In [1]:
import clr
clr.AddReference(r"..\bin\MiniSim.Core")
clr.AddReference(r"..\bin\MiniSim.FlowsheetDrawing")
from System import Enum
import MiniSim.Core.Expressions as expr
from  MiniSim.Core.Flowsheeting import MaterialStream, Flowsheet,IconTypes
from  MiniSim.Core.Flowsheeting.Documentation import SpreadsheetElement
import MiniSim.Core.Numerics as num
from MiniSim.Core.UnitsOfMeasure import Unit, SI, METRIC, PhysicalDimension
from MiniSim.Core.ModelLibrary import Flash, Heater, Mixer, Splitter, EquilibriumStageSection, BlackBoxReactor,Valve, FlowMode,ValveCharacteristic, NeuralNet
import MiniSim.Core.PropertyDatabase as chemsep
from MiniSim.Core.Reporting import Generator, StringBuilderLogger
from MiniSim.Core.Thermodynamics import ThermodynamicSystem
from MiniSim.FlowsheetDrawing import FlowsheetDrawer, DrawingOptions

In [2]:
logger = StringBuilderLogger();

In [3]:
def report(a,b,z, solver):
    for sys in solver.Subproblems:    
        print(f"Solve equation {sys.Equations[0]} for variable {sys.Variables[0]}")
    
    print(f"a={a.Val()}")
    print(f"b={b.Val()}")
    print(f"z={z.Val()}")


# Create a new Neural Net

In this example we create a very simple neural network with two inputs, a single hidden layer with 1 neuron and a single output. The weights are initialized as 1. The logistic function y=1/(1+exp(-u)) is used as the activation function. The bias is 0 for all neurons.

In [4]:
a = expr.Variable("a", 1.0)
b = expr.Variable("b", 1.0)
z = expr.Variable("z", 1.0)

net = NeuralNet("NN1", 2, 1, 1)

net.BindInput(0, a) \
    .BindInput(1, b)\
    .BindOutput(0, z)

solver =  num.DecompositionSolver(logger)
flowsheet = Flowsheet("Test: Neural Net")

flowsheet.AddCustomVariable(z)
flowsheet.AddUnit(net)

status = solver.Solve(flowsheet)
print(logger.Flush())
report(a,b,z,solver)

Decomposition Result: V=6, E=6, Blocks=6, Singletons=6
Block Statistics:
   # Var # Blocks % Blocks
       1        6 100,00 %
Problem NLAES was successfully solved (0,04 seconds)

Solve equation EQ000005 >> 0 := U[1] - b = 0 for variable U[1]
Solve equation EQ000004 >> 0 := U[0] - a = 0 for variable U[0]
Solve equation EQ000001 >> 0 := u[0,0] - ((1 * U[0] + 1 * U[1]) + 0) = 0 for variable u[0,0]
Solve equation EQ000002 >> 0 := y[0,0] - 1 / 1 + exp(-(u[0,0])) = 0 for variable y[0,0]
Solve equation EQ000003 >> 0 := Y[0] - (1 * y[0,0]) = 0 for variable Y[0]
Solve equation EQ000006 >> 0 := Y[0] - z = 0 for variable z
a=1.0
b=1.0
z=0.8807970779778823


In the output we see that the system was decomposed into 6 sub problems, each consisting of one variable and one equation. A perfect matching was found. The equations can be solved one after another, and then the result is used for the calculation of the next variable.

The neural net responds with an output z=0.881 for the inputs a=1 and b=1.

# Change Input Values

We change the value of the input variable a from 1.0 to 2.0 and observe the output.

In [5]:
a.Fix(2.0)
status = solver.Solve(flowsheet)
print(logger.Flush())
report(a,b,z,solver)

Decomposition Result: V=6, E=6, Blocks=6, Singletons=6
Block Statistics:
   # Var # Blocks % Blocks
       1        6 100,00 %
Problem NLAES was successfully solved (0,00 seconds)

Solve equation EQ000005 >> 0 := U[1] - b = 0 for variable U[1]
Solve equation EQ000004 >> 0 := U[0] - a = 0 for variable U[0]
Solve equation EQ000001 >> 0 := u[0,0] - ((1 * U[0] + 1 * U[1]) + 0) = 0 for variable u[0,0]
Solve equation EQ000002 >> 0 := y[0,0] - 1 / 1 + exp(-(u[0,0])) = 0 for variable y[0,0]
Solve equation EQ000003 >> 0 := Y[0] - (1 * y[0,0]) = 0 for variable Y[0]
Solve equation EQ000006 >> 0 := Y[0] - z = 0 for variable z
a=2.0
b=1.0
z=0.9525741268224334


Increasing the input a from 1.0 to 2.0 raised the output from z=0.881 to z=0.952.

# Change Input Specifications

We remove the output z from the list of calculated variables. We then fix it to an arbitrary value (0.9 in this case). We then unfix the input variable a and add it to the flowsheet as a custom variable.

In [6]:
flowsheet.RemoveCustomVariable(z)
z.Fix(0.9)
a.Unfix()
flowsheet.AddCustomVariable(a)
status = solver.Solve(flowsheet)
print(logger.Flush())
report(a,b,z,solver)

Decomposition Result: V=6, E=6, Blocks=6, Singletons=6
Block Statistics:
   # Var # Blocks % Blocks
       1        6 100,00 %
Problem NLAES was successfully solved (0,00 seconds)

Solve equation EQ000006 >> 0 := Y[0] - z = 0 for variable Y[0]
Solve equation EQ000003 >> 0 := Y[0] - (1 * y[0,0]) = 0 for variable y[0,0]
Solve equation EQ000002 >> 0 := y[0,0] - 1 / 1 + exp(-(u[0,0])) = 2,23403033849934E-08 for variable u[0,0]
Solve equation EQ000005 >> 0 := U[1] - b = 0 for variable U[1]
Solve equation EQ000001 >> 0 := u[0,0] - ((1 * U[0] + 1 * U[1]) + 0) = 0 for variable U[0]
Solve equation EQ000004 >> 0 := U[0] - a = 0 for variable a
a=1.1972243291106524
b=1.0
z=0.9


The calculation order was automatically updated by the Dulmage-Mendelsohn decomposition routine. It still solved all 6 equations sequentially, but now the input U[0] is solved in the last step to get the value of a.

To achieve the output value z=0.9, an input a=1.197 is needed.